In [ ]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers , Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data = np.load('cifar-10.npz')

x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']

In [ ]:
unique_labels = np.unique(y_train)
print("Unique train labels:", unique_labels)

In [ ]:
plt.figure(figsize=(15, 15))

for i in range(len(unique_labels)):  # In case there are fewer than 10 unique labels
    label = unique_labels[i]
    idx = np.where(y_train == label)[0][0]  # Get the index of the first occurrence of this label
    plt.subplot(5, 5, i+1)
    plt.imshow(x_train[idx])  # Display image corresponding to this unique label
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(label) 
plt.show()

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

plt.figure(figsize=(12,6))
plt.suptitle('First Image of Each Class')

for i, label in enumerate(unique_labels):
    index = np.where(y_train == label)[0][0]    # use your label array name
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[index])                      # use your image array name
    plt.title(f"Class: {class_names[label]}")
    plt.xticks([])
    plt.yticks([])

plt.show()

In [ ]:
max_v=max(x_test.max(),x_train.max())
x_train = x_train / max_v
x_test = x_test / max_v
print(max_v)

In [ ]:
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

In [ ]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    layers.Flatten(),

    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',  
    patience=3,
    restore_best_weights=True
)

history = model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=3, 
    callbacks=[early_stop],
    batch_size=32
)

In [ ]:
train_loss, train_acc = model.evaluate(x_train, y_train)
print("Train Accuracy:", train_acc)
print("Train Loss:", train_loss)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)

In [ ]:
n = random.randint(0,9999)
plt.imshow(x_test[n])
plt.show()

In [ ]:
predicted_value = model.predict(x_test)
print("The Image is: ", class_names[np.argmax(predicted_value[n])])

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()